In [1]:
import google.generativeai as genai
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
import os
import json
import pandas as pd
import traceback
from dotenv import load_dotenv
import PyPDF2

e:\MCQ_Generator\MCQ_Generator\env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
load_dotenv()
os.getenv("GOOGLE_API_KEY")
genai.configure(api_key=os.getenv("GOOGLE_API_KEY"))

In [3]:
llm=ChatGoogleGenerativeAI(model="gemini-pro",temperature=0.7)

In [4]:
llm

ChatGoogleGenerativeAI(model='gemini-pro', client= genai.GenerativeModel(
   model_name='models/gemini-pro',
   generation_config={}.
   safety_settings={}
))

In [5]:
with open("E:\MCQ_Generator\MCQ_Generator\Response.json","r") as f:
    RESPONSE_JSON=json.load(f)

In [6]:
print(RESPONSE_JSON)

{'1': {'no': '1', 'mcq': 'multiple choice questions', 'options': {'a': 'choice here', 'b': 'choice here', 'c': 'choice here', 'd': 'choice here'}, 'correct': 'correct answer'}, '2': {'no': '2', 'mcq': 'multiple choice questions', 'options': {'a': 'choice here', 'b': 'choice here', 'c': 'choice here', 'd': 'choice here'}, 'correct': 'correct answer'}, '3': {'no': '3', 'mcq': 'multiple choice questions', 'options': {'a': 'choice here', 'b': 'choice here', 'c': 'choice here', 'd': 'choice here'}, 'correct': 'correct answer'}}


In [7]:
TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below  and use it as a guide. \
Ensure to make {number} MCQs

{RESPONSE_JSON}

"""

In [8]:
quiz_generation_prompt=PromptTemplate(
    input_variables=["text","number","subject","tone","RESPONSE_JSON"],
    template=TEMPLATE
)

In [9]:
quiz_generation_prompt

PromptTemplate(input_variables=['RESPONSE_JSON', 'number', 'subject', 'text', 'tone'], template='\nText:{text}\nYou are an expert MCQ maker. Given the above text, it is your job to create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. \nMake sure the questions are not repeated and check all the questions to be conforming the text as well.\nMake sure to format your response like  RESPONSE_JSON below  and use it as a guide. Ensure to make {number} MCQs\n\n{RESPONSE_JSON}\n\n')

In [10]:
quiz_chain=LLMChain(llm=llm,prompt=quiz_generation_prompt,output_key="quiz",verbose=True)

In [11]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [12]:
TEMPLATE2

'\nYou are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. \nif the quiz is not at per with the cognitive and analytical abilities of the students,update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities\nQuiz_MCQs:\n{quiz}\n\nCheck from an expert English Writer of the above quiz:\n'

In [13]:
quiz_evaluation_prompt=PromptTemplate(input_variables=["subject", "quiz"], template=TEMPLATE2)

In [14]:
review_chain=LLMChain(llm=llm,prompt=quiz_evaluation_prompt,output_key="review",verbose=True)

In [15]:
generate_evaluate_chain=SequentialChain(chains=[quiz_chain, review_chain], input_variables=["text", "number", "subject", "tone", "RESPONSE_JSON"],output_variables=["quiz", "review"], verbose=True,)

In [16]:
PATH="E:\MCQ_Generator\MCQ_Generator\data.txt"

In [17]:
print(PATH)

E:\MCQ_Generator\MCQ_Generator\data.txt


In [18]:
with open(PATH,"r") as file:
    TEXT=file.read()

In [19]:
print(TEXT)

Recent progress in generative models have resulted in models that can produce realistic text, images and video that can potentially revolutionize the way humans work, create content and interact with machines. The workshop on Generative AI at AIMLSystems will focus on the entire life-cycle of building and deploying such Generative AI systems, including data collection and processing, developing systems and requisite infrastructure, applications it enables, and the ethics associated with such technology covering concerns related to fairness, transparency and accountability. We invite original, unpublished work on Artificial Intelligence with a focus on generative AI and their use cases. Specifically, the topics of interest include but are not limited to:

Systems, architecture and infrastructure for Generative AI
Machine learning and Modeling using LLMs and Diffusion models
Large Language Models and its applications
Multi-modal Generative AI and its applications
Gen AI based Plugins and

In [20]:
generate_evaluate_chain

SequentialChain(verbose=True, chains=[LLMChain(verbose=True, prompt=PromptTemplate(input_variables=['RESPONSE_JSON', 'number', 'subject', 'text', 'tone'], template='\nText:{text}\nYou are an expert MCQ maker. Given the above text, it is your job to create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. \nMake sure the questions are not repeated and check all the questions to be conforming the text as well.\nMake sure to format your response like  RESPONSE_JSON below  and use it as a guide. Ensure to make {number} MCQs\n\n{RESPONSE_JSON}\n\n'), llm=ChatGoogleGenerativeAI(model='gemini-pro', client= genai.GenerativeModel(
   model_name='models/gemini-pro',
   generation_config={}.
   safety_settings={}
)), output_key='quiz'), LLMChain(verbose=True, prompt=PromptTemplate(input_variables=['quiz', 'subject'], template='\nYou are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.You need to evaluate the complexit

In [21]:
print(TEXT)

Recent progress in generative models have resulted in models that can produce realistic text, images and video that can potentially revolutionize the way humans work, create content and interact with machines. The workshop on Generative AI at AIMLSystems will focus on the entire life-cycle of building and deploying such Generative AI systems, including data collection and processing, developing systems and requisite infrastructure, applications it enables, and the ethics associated with such technology covering concerns related to fairness, transparency and accountability. We invite original, unpublished work on Artificial Intelligence with a focus on generative AI and their use cases. Specifically, the topics of interest include but are not limited to:

Systems, architecture and infrastructure for Generative AI
Machine learning and Modeling using LLMs and Diffusion models
Large Language Models and its applications
Multi-modal Generative AI and its applications
Gen AI based Plugins and

In [22]:
TEXT
NUMBER=5
SUBJECT="AI"
TONE="Simple",
RESPONSE_JSON=RESPONSE_JSON

In [23]:
RESPONSE_JSON

{'1': {'no': '1',
  'mcq': 'multiple choice questions',
  'options': {'a': 'choice here',
   'b': 'choice here',
   'c': 'choice here',
   'd': 'choice here'},
  'correct': 'correct answer'},
 '2': {'no': '2',
  'mcq': 'multiple choice questions',
  'options': {'a': 'choice here',
   'b': 'choice here',
   'c': 'choice here',
   'd': 'choice here'},
  'correct': 'correct answer'},
 '3': {'no': '3',
  'mcq': 'multiple choice questions',
  'options': {'a': 'choice here',
   'b': 'choice here',
   'c': 'choice here',
   'd': 'choice here'},
  'correct': 'correct answer'}}

In [24]:
import json
ab= json.dumps(RESPONSE_JSON)

In [25]:
response=generate_evaluate_chain(
        {

        "text": TEXT,
        "number": NUMBER,
        "subject": SUBJECT,
        "tone": TONE,
        "RESPONSE_JSON": ab

        }
    )

e:\MCQ_Generator\MCQ_Generator\env\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(




> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:Recent progress in generative models have resulted in models that can produce realistic text, images and video that can potentially revolutionize the way humans work, create content and interact with machines. The workshop on Generative AI at AIMLSystems will focus on the entire life-cycle of building and deploying such Generative AI systems, including data collection and processing, developing systems and requisite infrastructure, applications it enables, and the ethics associated with such technology covering concerns related to fairness, transparency and accountability. We invite original, unpublished work on Artificial Intelligence with a focus on generative AI and their use cases. Specifically, the topics of interest include but are not limited to:

Systems, architecture and infrastructure for Generative AI
Machine learning and Modeling using LLMs and Diffusion models
Large 

In [26]:
reviews = response.get("review")

In [27]:
reviews

'**Complexity Analysis:**\n\nThe quiz questions are well-structured and cover various aspects of Generative AI. However, some questions may require a deeper understanding of the concepts, which might not be suitable for all students.\n\n**Questions to be Updated:**\n\n* Question 2: The topic "Natural Language Processing" is not directly related to Generative AI and should be replaced with a more relevant topic.\n* Question 5: The ethical concerns associated with Generative AI are not explicitly mentioned in the options. It should be updated to include specific concerns like bias, privacy, and misinformation.\n\n**Revised Quiz Questions:**\n\n2. Which topic is not included in the list of topics of interest for the workshop?\n   a. Architectures and Algorithms for Generative AI\n   b. Applications of Generative AI in various domains\n   c. Ethical considerations and challenges in Generative AI\n   d. Machine learning and Modeling using LLMs and Diffusion models\n\n5. What is one of the e

In [28]:
abcd=response.get("quiz")

In [29]:
start_index = abcd.find('{')
end_index = abcd.rfind('}') + 1
quiz = abcd[start_index:end_index]

In [30]:
quiz=json.loads(quiz)

In [31]:
quiz

{'1': {'no': '1',
  'mcq': 'What is the main focus of the workshop on Generative AI at AIMLSystems?',
  'options': {'a': 'The entire life-cycle of building and deploying Generative AI systems',
   'b': 'Data collection and processing',
   'c': 'Developing systems and requisite infrastructure',
   'd': 'Applications it enables'},
  'correct': 'a'},
 '2': {'no': '2',
  'mcq': 'Which topic is not included in the list of topics of interest for the workshop?',
  'options': {'a': 'Systems, architecture and infrastructure for Generative AI',
   'b': 'Machine learning and Modeling using LLMs and Diffusion models',
   'c': 'Large Language Models and its applications',
   'd': 'Natural Language Processing'},
  'correct': 'd'},
 '3': {'no': '3',
  'mcq': 'What is one of the applications of Multi-modal Generative AI?',
  'options': {'a': 'Image generation',
   'b': 'Text generation',
   'c': 'Video generation',
   'd': 'All of the above'},
  'correct': 'd'},
 '4': {'no': '4',
  'mcq': 'What is the

In [32]:
reviews

'**Complexity Analysis:**\n\nThe quiz questions are well-structured and cover various aspects of Generative AI. However, some questions may require a deeper understanding of the concepts, which might not be suitable for all students.\n\n**Questions to be Updated:**\n\n* Question 2: The topic "Natural Language Processing" is not directly related to Generative AI and should be replaced with a more relevant topic.\n* Question 5: The ethical concerns associated with Generative AI are not explicitly mentioned in the options. It should be updated to include specific concerns like bias, privacy, and misinformation.\n\n**Revised Quiz Questions:**\n\n2. Which topic is not included in the list of topics of interest for the workshop?\n   a. Architectures and Algorithms for Generative AI\n   b. Applications of Generative AI in various domains\n   c. Ethical considerations and challenges in Generative AI\n   d. Machine learning and Modeling using LLMs and Diffusion models\n\n5. What is one of the e

In [33]:
quiz_table_data = []
for key, value in quiz.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
            ]
        )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

In [34]:
quiz_table_data

[{'MCQ': 'What is the main focus of the workshop on Generative AI at AIMLSystems?',
  'Choices': 'a: The entire life-cycle of building and deploying Generative AI systems | b: Data collection and processing | c: Developing systems and requisite infrastructure | d: Applications it enables',
  'Correct': 'a'},
 {'MCQ': 'Which topic is not included in the list of topics of interest for the workshop?',
  'Choices': 'a: Systems, architecture and infrastructure for Generative AI | b: Machine learning and Modeling using LLMs and Diffusion models | c: Large Language Models and its applications | d: Natural Language Processing',
  'Correct': 'd'},
 {'MCQ': 'What is one of the applications of Multi-modal Generative AI?',
  'Choices': 'a: Image generation | b: Text generation | c: Video generation | d: All of the above',
  'Correct': 'd'},
 {'MCQ': 'What is the term used for Generative AI based Plugins and agents?',
  'Choices': 'a: Gen AI Plugins | b: Gen AI Agents | c: Both a and b | d: None of

In [35]:
df=pd.DataFrame(quiz_table_data)

In [36]:
df.to_csv("genai.csv",index=False)

In [37]:
LOG_FILE="{datetime.now().strftime('%m_%d_%Y_%H_%M_%S')}.log"

In [38]:
LOG_FILE

"{datetime.now().strftime('%m_%d_%Y_%H_%M_%S')}.log"

In [39]:
from datetime import datetime
datetime.now().strftime('%m_%d_%Y_%H_%M_%S')

'02_23_2024_15_14_53'